In [2]:
import requests, json

In [ ]:
curl --compressed -H 'Accept: application/json' -H 'X-Api-Key: <client-api-key>' \
  'https://<api_hostname>/<endpoint>'

In [ ]:
GET https://<eu|us>.market-api.kaiko.io/v1/data/trades.v1/spot_exchange_rate/{base_asset}/{quote_asset}{?exchanges,start_time,end_time,interval,page_size,sources}

In [ ]:
response = requests.get(url='https://us.market-api.kaiko.io/v1/data/trades.v1/spot_exchange_rate/btc/usd{?exchanges,start_time,end_time,interval,page_size,sources}')
json_data = json.loads(response.text)
json_data

In [ ]:
curl --compressed -H 'Accept: application/json' -H 'X-Api-Key: 5xzbwoyyz540wxsypgw27b7rttm1gvb7' \
  'https://us.market-api.kaiko.io/v1/data/trades.v1/spot_exchange_rate/btc/usd?start_time=2019-10-01T00:00:00Z&end_time=2019-10-03T00:00:00Z&exchanges=krkn&sources=true'

binance api key : Dlu2UuLy8o15VJoqpXj9ARx2N7OOkT03aDJUtYoTXvbXGP7VGJO5x1i5qOJiO5ZE

secret:um7cEJPwcAoJ1qYRUo8qBwdb42RNpToIN2vWhB7SPrBODDW91Hvu8058wNm2GIia

bitmex api key: zOXzXtq8pO-MU2gVriuaSzNe

secret: ziG8H08SvPTolY2qC878bPS9dEV8ag2DCuN4plcbWYTSQfNh

In [11]:
from binance.client import Client

ModuleNotFoundError: No module named 'binance.client'; 'binance' is not a package

## BTC Price extraction

In [1]:
import pandas as pd
import math
import os.path
import time
from bitmex import bitmex
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

### API
bitmex_api_key = 'zOXzXtq8pO-MU2gVriuaSzNe'    #Enter your own API-key here
bitmex_api_secret = 'ziG8H08SvPTolY2qC878bPS9dEV8ag2DCuN4plcbWYTSQfNh' #Enter your own API-secret here
binance_api_key = 'Dlu2UuLy8o15VJoqpXj9ARx2N7OOkT03aDJUtYoTXvbXGP7VGJO5x1i5qOJiO5ZE'    #Enter your own API-key here
binance_api_secret = 'um7cEJPwcAoJ1qYRUo8qBwdb42RNpToIN2vWhB7SPrBODDW91Hvu8058wNm2GIia' #Enter your own API-secret here

### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
bitmex_client = bitmex(test=False, api_key=bitmex_api_key, api_secret=bitmex_api_secret)
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)


### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Nov 2019', '%d %b %Y')
    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Nov 2019', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

def get_all_bitmex(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
    data_df.set_index('timestamp', inplace=True)
    if save and rounds > 0: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

/Users/msh/Library/Python/3.6/lib/python/site-packages/swagger_spec_validator/validator20.py:53: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  ref_dict['$ref'], '/'.join(path),


In [5]:
data = get_all_binance('BTCUSDT','1m', save = True)

KeyboardInterrupt: 

In [3]:
data = get_all_bitmex('XBTUSD', '1m', save = True)


All caught up..!


In [13]:
data

,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
timestamp,,,,,,,,,,,,
2015-09-25 12:01:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
2015-09-25 12:02:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
2015-09-25 12:03:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
2015-09-25 12:04:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
2015-09-25 12:05:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-16 16:18:00+00:00,XBTUSD,7017.5,7017.5,7006.0,7007.5,705,1188447,7011.1477,100.0,16951500267,169.515003,1188447.0
2020-04-16 16:19:00+00:00,XBTUSD,7007.5,7012.0,7007.5,7012.0,194,256923,7011.1477,600.0,3664686227,36.646862,256923.0
2020-04-16 16:20:00+00:00,XBTUSD,7012.0,7012.0,7011.5,7012.0,95,80188,7012.1310,100.0,1143603203,11.436032,80188.0


In [84]:
import pytz
utc=pytz.UTC

In [93]:
data_folder = '/Users/msh/Desktop/Thesis/data/'

In [91]:
# timestamp of the first data entry from exchanges, localized 
e = utc.localize(datetime.fromtimestamp(1569888001275/1000))

In [211]:
start_time_stamp = int(1569888001275/1000)
start_time_stamp

1569888001

In [181]:
data = pd.read_csv(data_folder+'XBTUSD-1m-data.csv')

In [187]:
data.head()

,timestamp,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
0,2015-09-25 12:01:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.0,0.0
1,2015-09-25 12:02:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.0,0.0
2,2015-09-25 12:03:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.0,0.0
3,2015-09-25 12:04:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.0,0.0
4,2015-09-25 12:05:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.0,0.0


In [188]:
2397862 - 6*30*24*60

2138662

In [195]:
time_data = data[2111662:]

In [197]:
time_data.head()

,timestamp,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
2111662,2019-09-30 22:23:00+00:00,XBTUSD,8262.0,8262.0,8257.5,8257.5,300,706176,8259.0023,1.0,8550986410,85.509864,706176.0
2111663,2019-09-30 22:24:00+00:00,XBTUSD,8257.5,8258.0,8257.5,8257.5,52,68768,8258.3203,1000.0,832756560,8.327566,68768.0
2111664,2019-09-30 22:25:00+00:00,XBTUSD,8257.5,8258.0,8257.0,8257.0,160,181976,8257.6383,2000.0,2203740227,22.037402,181976.0
2111665,2019-09-30 22:26:00+00:00,XBTUSD,8257.0,8257.5,8255.5,8256.0,259,425331,8256.9565,6000.0,5151368026,51.513680,425331.0
2111666,2019-09-30 22:27:00+00:00,XBTUSD,8256.0,8255.5,8248.0,8248.5,793,747859,8251.5059,8000.0,9064029972,90.640300,747859.0


In [198]:
for index, row in time_data.iterrows():
    date = row['timestamp']
    date = date[:-6]
    date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    ts = datetime.timestamp(date)
    ts = int(ts)
    time_data.at[index,'timestamp'] = ts

In [199]:
time_data.head()

,timestamp,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
2111662,1569871380,XBTUSD,8262.0,8262.0,8257.5,8257.5,300,706176,8259.0023,1.0,8550986410,85.509864,706176.0
2111663,1569871440,XBTUSD,8257.5,8258.0,8257.5,8257.5,52,68768,8258.3203,1000.0,832756560,8.327566,68768.0
2111664,1569871500,XBTUSD,8257.5,8258.0,8257.0,8257.0,160,181976,8257.6383,2000.0,2203740227,22.037402,181976.0
2111665,1569871560,XBTUSD,8257.0,8257.5,8255.5,8256.0,259,425331,8256.9565,6000.0,5151368026,51.513680,425331.0
2111666,1569871620,XBTUSD,8256.0,8255.5,8248.0,8248.5,793,747859,8251.5059,8000.0,9064029972,90.640300,747859.0


In [200]:
time_data.to_csv(data_folder+'XBTUSD-1m-data-from-30.09.2019.csv')

In [203]:
new_data = pd.read_csv(data_folder+'XBTUSD-1m-data-from-30.09.2019.csv',index_col='timestamp')

In [204]:
new_data.head()

,Unnamed: 0,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
timestamp,,,,,,,,,,,,,
1569871380,2111662,XBTUSD,8262.0,8262.0,8257.5,8257.5,300,706176,8259.0023,1.0,8550986410,85.509864,706176.0
1569871440,2111663,XBTUSD,8257.5,8258.0,8257.5,8257.5,52,68768,8258.3203,1000.0,832756560,8.327566,68768.0
1569871500,2111664,XBTUSD,8257.5,8258.0,8257.0,8257.0,160,181976,8257.6383,2000.0,2203740227,22.037402,181976.0
1569871560,2111665,XBTUSD,8257.0,8257.5,8255.5,8256.0,259,425331,8256.9565,6000.0,5151368026,51.513680,425331.0
1569871620,2111666,XBTUSD,8256.0,8255.5,8248.0,8248.5,793,747859,8251.5059,8000.0,9064029972,90.640300,747859.0


In [207]:
col_names = ['symbol', 'open', 'high', 'low', 'close', 'trades', 'volume', 'vwap', 'lastSize', 'turnover', 'homeNotional', 'foreignNotional']

In [220]:
new_data.loc[(new_data.index > start_time_stamp),col_names]

,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
timestamp,,,,,,,,,,,,
1569888060,XBTUSD,8423.5,8423.5,8410.5,8411.0,970,3418053,8418.2170,10500.0,40605562671,406.055627,3418053.0
1569888120,XBTUSD,8411.0,8420.0,8408.0,8420.0,740,2512725,8410.4289,4230.0,29876906316,298.769063,2512725.0
1569888180,XBTUSD,8420.0,8432.0,8419.5,8431.5,994,4457930,8428.8604,1000.0,52891337795,528.913378,4457930.0
1569888240,XBTUSD,8431.5,8432.0,8426.0,8430.0,564,1997199,8429.5709,3000.0,23693768003,236.937680,1997199.0
1569888300,XBTUSD,8430.0,8450.0,8429.5,8435.0,2700,17977446,8440.9555,50.0,212980372228,2129.803722,17977446.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1587043080,XBTUSD,7017.5,7017.5,7006.0,7007.5,705,1188447,7011.1477,100.0,16951500267,169.515003,1188447.0
1587043140,XBTUSD,7007.5,7012.0,7007.5,7012.0,194,256923,7011.1477,600.0,3664686227,36.646862,256923.0
1587043200,XBTUSD,7012.0,7012.0,7011.5,7012.0,95,80188,7012.1310,100.0,1143603203,11.436032,80188.0


In [221]:
new_data.to_csv(data_folder+'XBTUSD-1m-data-final.csv')

In [228]:
col_names = ['timestamp','symbol', 'open', 'high', 'low', 'close', 'trades', 'volume', 'vwap', 'lastSize', 'turnover', 'homeNotional', 'foreignNotional']

In [229]:
check = pd.read_csv(data_folder+'XBTUSD-1m-data-final.csv',index_col='timestamp',usecols=col_names)

In [230]:
check.head()

,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
timestamp,,,,,,,,,,,,
1569871380,XBTUSD,8262.0,8262.0,8257.5,8257.5,300,706176,8259.0023,1.0,8550986410,85.509864,706176.0
1569871440,XBTUSD,8257.5,8258.0,8257.5,8257.5,52,68768,8258.3203,1000.0,832756560,8.327566,68768.0
1569871500,XBTUSD,8257.5,8258.0,8257.0,8257.0,160,181976,8257.6383,2000.0,2203740227,22.037402,181976.0
1569871560,XBTUSD,8257.0,8257.5,8255.5,8256.0,259,425331,8256.9565,6000.0,5151368026,51.513680,425331.0
1569871620,XBTUSD,8256.0,8255.5,8248.0,8248.5,793,747859,8251.5059,8000.0,9064029972,90.640300,747859.0


# Exchanges Data Extraction

In [4]:
import os
import pandas as pd
import numpy as np

In [234]:
kaiko_data_link = '/Users/msh/Desktop/Thesis/data/kaiko-ob10/monthly_tar_v1/'

In [240]:
s = os.listdir(kaiko_data_link)[1]

In [242]:
s.endswith('as')

False

In [237]:
for d in os.listdir(kaiko_data_link):
    if d != '.DS_Store':

['.DS_Store', '11', '10', '12']
['Bitstamp_BTCUSD_ob_10_2020_01', '.DS_Store', 'Kraken_BTCUSDC_ob_10_2020_01.tar', 'Gemini_BTCUSD_ob_10_2020_01.tar', 'Kraken_BTCUSD_ob_10_2020_01.tar', 'Kraken_BTCUSD_ob_10_2020_01', 'Kraken_BTCUSDT_ob_10_2020_01.tar', 'Bitstamp_BTCUSD_ob_10_2020_01.tar', 'Gemini_BTCUSD_ob_10_2020_01']


In [246]:
def parse_month(link_to_folder):
    folder_names = os.listdir(link_to_folder)
    folder_names.remove('.DS_Store')
    names = [name for name in folder_names if name.endswith('.tar') is False]
    return names

In [247]:
parse_month('/Users/msh/Desktop/Thesis/data/kaiko-ob10/monthly_tar_v1/2019/10')

['Bitstamp_BTCUSD_ob_10_2019_10',
 'Kraken_BTCUSD_ob_10_2019_10',
 'Gemini_BTCUSD_ob_10_2019_10']

In [1]:
test_link = '/Users/msh/Desktop/Thesis/data/exchanges/kaiko-ob10/monthly_tar_v1/2019/10/Bitstamp_BTCUSD_ob_10_2019_10/Bitstamp_BTCUSD_ob_10_2019_10_01.csv'

In [10]:
df = pd.read_csv(test_link)

In [11]:
df.head()

,date,type,price,amount
0,1569888000398,b,8300.50,0.600000
1,1569888000398,b,8297.57,0.500000
2,1569888000398,b,8297.56,2.099967
3,1569888000398,b,8297.54,2.000000
4,1569888000398,b,8297.53,2.500000


In [12]:
timestamps = df.date.unique()

In [13]:
len(timestamps)

2880

In [14]:
timestamps[3] - timestamps[4]

-59121

In [15]:
timestamps[1]

1569888001250

In [ ]:
# def filter_timestaps(timestamps, filter_value):
#     final = []
#     for (i = 0; i < len(timestamps) - 2; i+2):
#         if np.abs(timestamps[i] - timestamps[i+1]) > filter_value:

In [17]:
2%2

0

In [16]:
def filter_timestaps(timestamps):
    final = []
    for idx, timestamp in enumerate(timestamps):
        if idx % 2 == 0:
            final.append(timestamp)
    return final

In [17]:
filtered_timestamps = filter_timestaps(timestamps)

In [18]:
len(filtered_timestamps) # 1440 = 60*24

1440

In [24]:
clear_pd = df[df.date.isin(filtered_timestamps)]

In [25]:
clear_pd

,date,type,price,amount
0,1569888000398,b,8300.50,0.600000
1,1569888000398,b,8297.57,0.500000
2,1569888000398,b,8297.56,2.099967
3,1569888000398,b,8297.54,2.000000
4,1569888000398,b,8297.53,2.500000
...,...,...,...,...
2495452,1569974341381,a,9149.00,0.020000
2495453,1569974341381,a,9149.08,0.004374
2495454,1569974341381,a,9150.00,0.320000
2495455,1569974341381,a,9152.00,0.060000


In [20]:
len(clear_pd.date.unique())

1440

In [23]:
df[(df.date==1569888001250)]

,date,type,price,amount
897,1569888001250,b,8298.45,0.596985
898,1569888001250,b,8298.11,2.000000
899,1569888001250,b,8297.56,2.099967
900,1569888001250,b,8297.54,2.000000
901,1569888001250,b,8297.53,2.500000
...,...,...,...,...
1792,1569888001250,a,9119.53,12.500000
1793,1569888001250,a,9127.00,0.020000
1794,1569888001250,a,9132.03,0.052354
1795,1569888001250,a,9133.33,0.100000


In [26]:
clear_pd[(clear_pd.date==1569888001250)]

,date,type,price,amount


In [55]:
def get_bids_asks(dataframe):
    bids, asks = [], []
    for index, data in dataframe.iterrows():
        if data.type == 'b':
            bids.append((data.price, data.amount))
        if data.type == 'a':
            asks.append((data.price, data.amount))
    return bids, asks    

In [56]:
bids, asks = get_bids_asks(clear_pd[(clear_pd.date==filtered_timestamps[0])])

In [59]:
len(bids) + len(asks)

897

In [60]:
def calculate_VWAP(bids_or_asks):
    numerator = 0.0
    denominator = 0.0
    for (price, volume) in bids_or_asks:
        numerator += price * volume
        denominator += volume
    return numerator / denominator

In [70]:
VWAP_bids = calculate_VWAP(bids)
VWAP_bids

7801.374018550332

In [71]:
VWAP_asks = calculate_VWAP(asks)
VWAP_asks

8613.916464929336

In [65]:
def calculate_Midpoint(bids,asks):
    return (bids[0][0] + asks[0][0])/ 2

In [69]:
Midpoint = calculate_Midpoint(bids,asks)
Midpoint

8304.975

In [72]:
def calculate_VWAP_Midpoint(VWAP_bids,VWAP_asks):
    return (VWAP_bids + VWAP_asks) / 2

In [74]:
VWAP_Midpoint = calculate_VWAP_Midpoint(VWAP_bids,VWAP_asks)
VWAP_Midpoint

8207.645241739834

In [132]:
def process_dataframe(link_to_csv):
    
    def filter_timestaps(timestamps):
        final = []
        for idx, timestamp in enumerate(timestamps):
            if idx % 2 == 0:
                final.append(timestamp)
        return final

    def get_bids_asks(dataframe):
        bids, asks = [], []
        for index, data in dataframe.iterrows():
            if data.type == 'b':
                bids.append((data.price, data.amount))
            if data.type == 'a':
                asks.append((data.price, data.amount))
        return bids, asks    
    
    def calculate_VWAP(bids_or_asks):
        numerator = 0.0
        denominator = 0.0
        for (price, volume) in bids_or_asks:
            numerator += price * volume
            denominator += volume
        return numerator / denominator

    def calculate_midpoint(bids,asks):
        return (bids[0][0] + asks[0][0])/ 2
    
    def calculate_VWAP_midpoint(VWAP_bids,VWAP_asks):
        return (VWAP_bids + VWAP_asks) / 2
    
    df = pd.read_csv(link_to_csv)
    timestamps = df.date.unique()
    filtered_timestamps = filter_timestaps(timestamps)
    
    data = []
    columns = ['timestamp','bids_VWAP','asks_VWAP','midpoint','VWAP_midpoint']
    
    for timestamp in filtered_timestamps:
        frame = df[(df.date==timestamp)]
        bids, asks = get_bids_asks(frame)
        bids_VWAP = calculate_VWAP(bids)
        asks_VWAP = calculate_VWAP(asks)
        midpoint = calculate_midpoint(bids, asks)
        VWAP_midpoint = calculate_VWAP_midpoint(bids_VWAP, asks_VWAP)
        data.append(list((timestamp,bids_VWAP,asks_VWAP,midpoint,VWAP_midpoint)))
    
    data_frame = pd.DataFrame(data=list(data),columns=columns)
    data_frame = data_frame.set_index('timestamp')
        
    return (data_frame)

In [133]:
result_final = process_dataframe(test_link)

In [134]:
result_final

,bids_VWAP,asks_VWAP,midpoint,VWAP_midpoint
timestamp,,,,
1569888000398,7801.374019,8613.916465,8304.975,8207.645242
1569888060403,7794.424943,8609.953525,8305.975,8202.189234
1569888120398,7789.463626,8607.403914,8297.230,8198.433770
1569888180398,7786.937094,8607.873367,8288.435,8197.405230
1569888240399,7777.613453,8582.349956,8266.495,8179.981704
...,...,...,...,...
1569974101389,7847.620250,8630.287560,8317.150,8238.953905
1569974161386,7845.722810,8624.958728,8313.310,8235.340769
1569974221382,7846.170470,8624.792286,8313.620,8235.481378


In [136]:
def save_dataframe(link_to_folder, name, dataframe):
    dataframe.to_pickle(link_to_folder + name + '.p')

In [138]:
save_dataframe(clear_data_link, 'Bitstamp_BTCUSD_ob_10_2019_10_01', result_final)

In [139]:
teeee = pd.read_pickle(clear_data_link+'Bitstamp_BTCUSD_ob_10_2019_10_01'+'.p')

In [140]:
teeee

,bids_VWAP,asks_VWAP,midpoint,VWAP_midpoint
timestamp,,,,
1569888000398,7801.374019,8613.916465,8304.975,8207.645242
1569888060403,7794.424943,8609.953525,8305.975,8202.189234
1569888120398,7789.463626,8607.403914,8297.230,8198.433770
1569888180398,7786.937094,8607.873367,8288.435,8197.405230
1569888240399,7777.613453,8582.349956,8266.495,8179.981704
...,...,...,...,...
1569974101389,7847.620250,8630.287560,8317.150,8238.953905
1569974161386,7845.722810,8624.958728,8313.310,8235.340769
1569974221382,7846.170470,8624.792286,8313.620,8235.481378


In [181]:
dir_link = '/Users/msh/Desktop/Thesis/data/exchanges/kaiko-ob10/monthly_tar_v1/2019/10/Kraken_BTCUSD_ob_10_2019_10/'

In [182]:
links = os.listdir(dir_link)

In [183]:
links = [link for link in links if link.endswith('.csv')]

In [184]:
links = sorted(links)
links

['Kraken_BTCUSD_ob_10_2019_10_01.csv',
 'Kraken_BTCUSD_ob_10_2019_10_02.csv',
 'Kraken_BTCUSD_ob_10_2019_10_03.csv',
 'Kraken_BTCUSD_ob_10_2019_10_04.csv',
 'Kraken_BTCUSD_ob_10_2019_10_05.csv',
 'Kraken_BTCUSD_ob_10_2019_10_06.csv',
 'Kraken_BTCUSD_ob_10_2019_10_07.csv',
 'Kraken_BTCUSD_ob_10_2019_10_08.csv',
 'Kraken_BTCUSD_ob_10_2019_10_09.csv',
 'Kraken_BTCUSD_ob_10_2019_10_10.csv',
 'Kraken_BTCUSD_ob_10_2019_10_11.csv',
 'Kraken_BTCUSD_ob_10_2019_10_12.csv',
 'Kraken_BTCUSD_ob_10_2019_10_13.csv',
 'Kraken_BTCUSD_ob_10_2019_10_14.csv',
 'Kraken_BTCUSD_ob_10_2019_10_15.csv',
 'Kraken_BTCUSD_ob_10_2019_10_16.csv',
 'Kraken_BTCUSD_ob_10_2019_10_17.csv',
 'Kraken_BTCUSD_ob_10_2019_10_18.csv',
 'Kraken_BTCUSD_ob_10_2019_10_19.csv',
 'Kraken_BTCUSD_ob_10_2019_10_20.csv',
 'Kraken_BTCUSD_ob_10_2019_10_21.csv',
 'Kraken_BTCUSD_ob_10_2019_10_22.csv',
 'Kraken_BTCUSD_ob_10_2019_10_23.csv',
 'Kraken_BTCUSD_ob_10_2019_10_24.csv',
 'Kraken_BTCUSD_ob_10_2019_10_25.csv',
 'Kraken_BTCUSD_ob_10_201

In [185]:
names = [name[:-4] for name in links]
names

['Kraken_BTCUSD_ob_10_2019_10_01',
 'Kraken_BTCUSD_ob_10_2019_10_02',
 'Kraken_BTCUSD_ob_10_2019_10_03',
 'Kraken_BTCUSD_ob_10_2019_10_04',
 'Kraken_BTCUSD_ob_10_2019_10_05',
 'Kraken_BTCUSD_ob_10_2019_10_06',
 'Kraken_BTCUSD_ob_10_2019_10_07',
 'Kraken_BTCUSD_ob_10_2019_10_08',
 'Kraken_BTCUSD_ob_10_2019_10_09',
 'Kraken_BTCUSD_ob_10_2019_10_10',
 'Kraken_BTCUSD_ob_10_2019_10_11',
 'Kraken_BTCUSD_ob_10_2019_10_12',
 'Kraken_BTCUSD_ob_10_2019_10_13',
 'Kraken_BTCUSD_ob_10_2019_10_14',
 'Kraken_BTCUSD_ob_10_2019_10_15',
 'Kraken_BTCUSD_ob_10_2019_10_16',
 'Kraken_BTCUSD_ob_10_2019_10_17',
 'Kraken_BTCUSD_ob_10_2019_10_18',
 'Kraken_BTCUSD_ob_10_2019_10_19',
 'Kraken_BTCUSD_ob_10_2019_10_20',
 'Kraken_BTCUSD_ob_10_2019_10_21',
 'Kraken_BTCUSD_ob_10_2019_10_22',
 'Kraken_BTCUSD_ob_10_2019_10_23',
 'Kraken_BTCUSD_ob_10_2019_10_24',
 'Kraken_BTCUSD_ob_10_2019_10_25',
 'Kraken_BTCUSD_ob_10_2019_10_26',
 'Kraken_BTCUSD_ob_10_2019_10_27',
 'Kraken_BTCUSD_ob_10_2019_10_28',
 'Kraken_BTCUSD_ob_1

In [186]:
clear_datafolder_link = '/Users/msh/Desktop/Thesis/data/exchanges/clear/kraken/'

In [187]:
for idx, name in enumerate(names):
    link_to_csv = dir_link + links[idx]
    processed_dataframe = process_dataframe(link_to_csv)
    save_dataframe(clear_datafolder_link,name,processed_dataframe)

In [188]:
print('a')

a


In [189]:
dir_link = '/Users/msh/Desktop/Thesis/data/exchanges/kaiko-ob10/monthly_tar_v1/2019/11/Bitstamp_BTCUSD_ob_10_2019_11/'

links = os.listdir(dir_link)

links = [link for link in links if link.endswith('.csv')]

links = sorted(links)

names = [name[:-4] for name in links]

clear_datafolder_link = '/Users/msh/Desktop/Thesis/data/exchanges/clear/bitstamp/'

for idx, name in enumerate(names):
    link_to_csv = dir_link + links[idx]
    processed_dataframe = process_dataframe(link_to_csv)
    save_dataframe(clear_datafolder_link,name,processed_dataframe)

In [190]:
dir_link = '/Users/msh/Desktop/Thesis/data/exchanges/kaiko-ob10/monthly_tar_v1/2019/11/Gemini_BTCUSD_ob_10_2019_11/'

links = os.listdir(dir_link)

links = [link for link in links if link.endswith('.csv')]

links = sorted(links)

names = [name[:-4] for name in links]

clear_datafolder_link = '/Users/msh/Desktop/Thesis/data/exchanges/clear/gemini/'

for idx, name in enumerate(names):
    link_to_csv = dir_link + links[idx]
    processed_dataframe = process_dataframe(link_to_csv)
    save_dataframe(clear_datafolder_link,name,processed_dataframe)

In [191]:
dir_link = '/Users/msh/Desktop/Thesis/data/exchanges/kaiko-ob10/monthly_tar_v1/2019/11/Kraken_BTCUSD_ob_10_2019_11/'

links = os.listdir(dir_link)

links = [link for link in links if link.endswith('.csv')]

links = sorted(links)

names = [name[:-4] for name in links]

clear_datafolder_link = '/Users/msh/Desktop/Thesis/data/exchanges/clear/kraken/'

for idx, name in enumerate(names):
    link_to_csv = dir_link + links[idx]
    processed_dataframe = process_dataframe(link_to_csv)
    save_dataframe(clear_datafolder_link,name,processed_dataframe)

In [192]:
dir_link = '/Users/msh/Desktop/Thesis/data/exchanges/kaiko-ob10/monthly_tar_v1/2019/12/Bitstamp_BTCUSD_ob_10_2019_12/'

links = os.listdir(dir_link)

links = [link for link in links if link.endswith('.csv')]

links = sorted(links)

names = [name[:-4] for name in links]

clear_datafolder_link = '/Users/msh/Desktop/Thesis/data/exchanges/clear/bitstamp/'

for idx, name in enumerate(names):
    link_to_csv = dir_link + links[idx]
    processed_dataframe = process_dataframe(link_to_csv)
    save_dataframe(clear_datafolder_link,name,processed_dataframe)

In [193]:
dir_link = '/Users/msh/Desktop/Thesis/data/exchanges/kaiko-ob10/monthly_tar_v1/2019/12/Gemini_BTCUSD_ob_10_2019_12/'

links = os.listdir(dir_link)

links = [link for link in links if link.endswith('.csv')]

links = sorted(links)

names = [name[:-4] for name in links]

clear_datafolder_link = '/Users/msh/Desktop/Thesis/data/exchanges/clear/gemini/'

for idx, name in enumerate(names):
    link_to_csv = dir_link + links[idx]
    processed_dataframe = process_dataframe(link_to_csv)
    save_dataframe(clear_datafolder_link,name,processed_dataframe)

In [194]:
dir_link = '/Users/msh/Desktop/Thesis/data/exchanges/kaiko-ob10/monthly_tar_v1/2019/12/Kraken_BTCUSD_ob_10_2019_12/'

links = os.listdir(dir_link)

links = [link for link in links if link.endswith('.csv')]

links = sorted(links)

names = [name[:-4] for name in links]

clear_datafolder_link = '/Users/msh/Desktop/Thesis/data/exchanges/clear/kraken/'

for idx, name in enumerate(names):
    link_to_csv = dir_link + links[idx]
    processed_dataframe = process_dataframe(link_to_csv)
    save_dataframe(clear_datafolder_link,name,processed_dataframe)

In [195]:
dir_link = '/Users/msh/Desktop/Thesis/data/exchanges/kaiko-ob10/monthly_tar_v1/2020/Bitstamp_BTCUSD_ob_10_2020_01/'

links = os.listdir(dir_link)

links = [link for link in links if link.endswith('.csv')]

links = sorted(links)

names = [name[:-4] for name in links]

clear_datafolder_link = '/Users/msh/Desktop/Thesis/data/exchanges/clear/bitstamp/'

for idx, name in enumerate(names):
    link_to_csv = dir_link + links[idx]
    processed_dataframe = process_dataframe(link_to_csv)
    save_dataframe(clear_datafolder_link,name,processed_dataframe)

In [196]:
dir_link = '/Users/msh/Desktop/Thesis/data/exchanges/kaiko-ob10/monthly_tar_v1/2020/Gemini_BTCUSD_ob_10_2020_01/'

links = os.listdir(dir_link)

links = [link for link in links if link.endswith('.csv')]

links = sorted(links)

names = [name[:-4] for name in links]

clear_datafolder_link = '/Users/msh/Desktop/Thesis/data/exchanges/clear/gemini/'

for idx, name in enumerate(names):
    link_to_csv = dir_link + links[idx]
    processed_dataframe = process_dataframe(link_to_csv)
    save_dataframe(clear_datafolder_link,name,processed_dataframe)

In [197]:
dir_link = '/Users/msh/Desktop/Thesis/data/exchanges/kaiko-ob10/monthly_tar_v1/2020/Kraken_BTCUSD_ob_10_2020_01/'

links = os.listdir(dir_link)

links = [link for link in links if link.endswith('.csv')]

links = sorted(links)

names = [name[:-4] for name in links]

clear_datafolder_link = '/Users/msh/Desktop/Thesis/data/exchanges/clear/kraken/'

for idx, name in enumerate(names):
    link_to_csv = dir_link + links[idx]
    processed_dataframe = process_dataframe(link_to_csv)
    save_dataframe(clear_datafolder_link,name,processed_dataframe)

In [202]:
opa = pd.read_pickle('/Users/msh/Desktop/Thesis/data/exchanges/clear/gemini/Gemini_BTCUSD_ob_10_2020_01_05.p')

In [203]:
opa

,bids_VWAP,asks_VWAP,midpoint,VWAP_midpoint
timestamp,,,,
1578182401778,7268.573715,7390.017579,7352.815,7329.295647
1578182461779,7278.926142,7399.331981,7352.985,7339.129061
1578182521812,7280.967974,7406.742454,7359.830,7343.855214
1578182581779,7290.192242,7406.612219,7368.485,7348.402230
1578182641786,7291.992752,7408.093418,7368.650,7350.043085
...,...,...,...,...
1578268510374,7302.202554,7397.979771,7343.070,7350.091162
1578268570374,7302.160537,7397.828493,7343.065,7349.994515
1578268630386,7306.173468,7413.143213,7349.725,7359.658341


# Blockchain Data Extraction

In [204]:
path_to_blockchain_folder = '/Users/msh/Desktop/Thesis/data/blockchain/'

In [205]:
os.listdir(path_to_blockchain_folder)

['blockchair_bitcoin_blocks_20191128.tsv',
 'blockchair_bitcoin_blocks_20191114.tsv',
 'blockchair_bitcoin_blocks_20200104.tsv',
 'blockchair_bitcoin_blocks_20200110.tsv',
 'blockchair_bitcoin_blocks_20200111.tsv',
 'blockchair_bitcoin_blocks_20200105.tsv',
 'blockchair_bitcoin_blocks_20191101.tsv',
 'blockchair_bitcoin_blocks_20191115.tsv',
 'blockchair_bitcoin_blocks_20191129.tsv',
 'blockchair_bitcoin_blocks_20191103.tsv',
 'blockchair_bitcoin_blocks_20191117.tsv',
 'blockchair_bitcoin_blocks_20200113.tsv',
 'blockchair_bitcoin_blocks_20200107.tsv',
 'blockchair_bitcoin_blocks_20200106.tsv',
 'blockchair_bitcoin_blocks_20200112.tsv',
 'blockchair_bitcoin_blocks_20191116.tsv',
 'blockchair_bitcoin_blocks_20191102.tsv',
 'blockchair_bitcoin_blocks_20191106.tsv',
 'blockchair_bitcoin_blocks_20191112.tsv',
 'blockchair_bitcoin_blocks_20200116.tsv',
 'blockchair_bitcoin_blocks_20200102.tsv',
 '.DS_Store',
 'blockchair_bitcoin_blocks_20200103.tsv',
 'blockchair_bitcoin_blocks_20200117.tsv

In [208]:
blockchain_cols = ['id','time','median_time','size','weight','fee_per_kb','fee_per_kb_usd']

In [207]:
pd.read_csv('/Users/msh/Desktop/Thesis/data/blockchain/blockchair_bitcoin_blocks_20191001.tsv',sep='\t',usecols=blockchain_cols)

,id,hash,time,median_time,size,stripped_size,weight,version,version_hex,version_bits,...,fee_per_kb,fee_per_kb_usd,fee_per_kwu,fee_per_kwu_usd,cdd_total,generation,generation_usd,reward,reward_usd,guessed_miner
0,597318,0000000000000000000c1f84a78143078f4420da7a6b7f...,2019-10-01 00:06:17,2019-09-30 23:07:10,1312743,893443,3993072,541065216,20400000,100000010000000000000000000000,...,44502.0,3.6844,14615.10,1.2100,8767.646008,1250000000,103488.375,1308403079,108323.6094,BTC.com
1,597319,0000000000000000000ca77f1894b19d6fd236fb3ed3cb...,2019-10-01 00:13:31,2019-09-30 23:08:08,1301817,897174,3993339,1073725440,3fffc000,111111111111111100000000000000,...,24599.6,2.0367,8015.87,0.6637,24709.513423,1250000000,103488.375,1282014918,106138.9141,Unknown
2,597320,00000000000000000002b2f4a53d5376bcdf8350a08484...,2019-10-01 00:47:03,2019-09-30 23:12:50,1248124,916876,3998752,536870912,20000000,100000000000000000000000000000,...,46909.5,3.8837,14632.70,1.2115,35857.026359,1250000000,103488.375,1308527346,108333.8984,F2Pool
3,597321,0000000000000000000233c444a9e1d7b73c3a0262cd6a...,2019-10-01 00:51:38,2019-09-30 23:35:06,1253401,913328,3993385,545259520,20800000,100000100000000000000000000000,...,29097.1,2.4090,9125.12,0.7555,16877.666512,1250000000,103488.375,1286458851,106506.8359,Unknown
4,597322,00000000000000000002ead46f40a694ef0bbda8c0c7ea...,2019-10-01 00:59:52,2019-09-30 23:41:48,1169434,941220,3993094,1073676288,3fff0000,111111111111110000000000000000,...,19614.0,1.6239,5741.87,0.4754,54626.926631,1250000000,103488.375,1272929616,105386.7344,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,597456,00000000000000000002059ddd10afb5209d478b8069f5...,2019-10-01 23:04:36,2019-10-01 22:05:21,705863,455879,2073500,536870912,20000000,100000000000000000000000000000,...,12421.4,1.0283,4221.32,0.3495,84114.460077,1250000000,103488.375,1258763113,104213.8828,AntPool
139,597457,000000000000000000003422f90cedee5d2d769879e8c3...,2019-10-01 23:22:22,2019-10-01 22:20:08,1339133,884684,3993185,536870912,20000000,100000000000000000000000000000,...,21125.3,1.7490,7075.55,0.5858,56085.800894,1250000000,103488.375,1278282655,105829.9219,BTC.com
140,597458,000000000000000000015449a5fe6f40509fd85ae18e5a...,2019-10-01 23:48:26,2019-10-01 22:22:38,1325812,889085,3993067,536870912,20000000,100000000000000000000000000000,...,31543.7,2.6115,10462.10,0.8662,40364.728546,1250000000,103488.375,1291808549,106949.7344,ViaBTC
141,597459,0000000000000000000cb4158b06b297e3432773baa0d6...,2019-10-01 23:52:21,2019-10-01 22:54:38,933112,600366,2734210,536870912,20000000,100000000000000000000000000000,...,10318.5,0.8543,3515.37,0.2910,1931.792213,1250000000,103488.375,1259624284,104285.1797,ViaBTC
